In [1]:
import os
from typing import TypedDict, Annotated, List
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver

c:\Users\anand\Desktop\perplexiy\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
load_dotenv()

True

In [3]:
# Agent State
# ---------------------------------------------------------
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

# ---------------------------------------------------------

In [4]:
# LLM Setup
# ---------------------------------------------------------
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.2,
    convert_system_message_to_human=True,
)

In [5]:
# Tools
# ---------------------------------------------------------
search_tool = TavilySearchResults(max_results=2)
search_tool.name = "tavily_search"

C:\Users\anand\AppData\Local\Temp\ipykernel_119120\3241713405.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=2)


In [6]:
tools = [search_tool]
llm_with_tools = llm.bind_tools(tools)

In [7]:
# Agent Node
# ---------------------------------------------------------
def agent_node(state: AgentState):
    from datetime import datetime
    current_date = datetime.now().strftime("%Y-%m-%d")
    
    system_prompt = SystemMessage(f"""
You are a helpful AI assistant. Today's date is {current_date}.

- For casual chat, answer normally.
- For factual questions, news, or anything requiring updated information,
  CALL the `tavily_search` tool.
""")

    messages = [system_prompt] + state["messages"]
    ai_reply = llm_with_tools.invoke(messages)

    return {"messages": [ai_reply]}


In [8]:
# ---------------------------------------------------------
# Should Continue (Tool or End)
# ---------------------------------------------------------
def should_continue(state: AgentState):
    last = state["messages"][-1]
    return "tools" if last.tool_calls else "__end__"


In [ ]:
# ---------------------------------------------------------
# Create Graph
# ---------------------------------------------------------

graph = StateGraph(AgentState)

graph.add_node("agent", agent_node)
graph.add_node("tools", ToolNode(tools))

graph.set_entry_point("agent")

graph.add_conditional_edges(
        "agent",
        should_continue,
        {
            "tools": "tools",
            "__end__": END
        }
    )

graph.add_edge("tools", "agent")

# Initialize MemorySaver
memory = MemorySaver()

# Compile with checkpointer
app = graph.compile(checkpointer=memory)

# Interactive Loop
print("Bot is ready! Type 'exit' to stop.")
config = {"configurable": {"thread_id": "1"}}

while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Goodbye!")
        break
    
    response = app.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config=config
    )
    
    print("Agent:", response["messages"][-1].content)


Bot is ready! Type 'exit' to stop.
Agent: Hello! How can I help you today?
